In [25]:
import yaml 
import requests
import pandas as pd
import sqlite3
from tqdm import tqdm
from bs4 import BeautifulSoup
from fpdf import FPDF
conn = sqlite3.connect("data/database.db")

In [26]:
tickers = pd.read_sql_query("SELECT * FROM master_ticker", conn)["Security"].drop_duplicates().tolist()
tickers = [t.replace(" ", "_") for t in tickers]

In [27]:
def create_pdf(text, filename="output.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin = 15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    # Split text into lines that fit in the PDF
    lines = pdf.multi_cell(0, 10, text)

    pdf.output(filename)
    print(f"PDF saved as {filename}")

In [ ]:
def extract_text_from_wikipedia(ticker):
    URL_BASE = "https://en.wikipedia.org/wiki/"
    request = requests.get(URL_BASE + ticker)
    soup = BeautifulSoup(request.text, "html.parser")
    pars = []
    for p in soup.find_all('p'):
        pars.append(str(p.text))
    
    text = "".join(pars).encode('latin-1', 'replace').decode('latin-1')
    print(text)
    
    create_pdf(text, filename="data/wikipedia/"+ str(ticker)+".pdf")

In [ ]:
class WikipediaPDFGenerator:
    def __init__(self, output_dir="data/wikipedia/"):
        self.output_dir = output_dir
    
    def create_pdf(self, text, filename="output.pdf"):
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        # Split text into lines that fit in the PDF
        pdf.multi_cell(0, 10, text)
        full_path = f"{self.output_dir}{filename}"
        pdf.output(full_path)
        print(f"PDF saved as {full_path}")

    def extract_text_from_wikipedia(self, ticker):
        URL_BASE = "https://en.wikipedia.org/wiki/"
        request = requests.get(URL_BASE + ticker)
        soup = BeautifulSoup(request.text, "html.parser")
        pars = [p.text for p in soup.find_all('p')]
        text = "".join(pars).encode('latin-1', 'replace').decode('latin-1')
        print(text)
        self.create_pdf(text, filename=f"{ticker}.pdf")

In [ ]:
generator = WikipediaPDFGenerator()
for t in tickers:
    generator.extract_text_from_wikipedia(ticker=t)


3M Company (originally the Minnesota Mining and Manufacturing Company) is an American multinational conglomerate operating in the fields of industry, worker safety, and consumer goods.[5] The company produces over 60,000 products under several brands,[6] including adhesives, abrasives, laminates, passive fire protection, personal protective equipment, window films, paint protection film, electrical, electronic connecting, insulating materials, car-care products,[7] electronic circuits, and optical films.[8] It is based in Maplewood, a suburb of Saint Paul, Minnesota.[9]
3M made $35.4 billion in total sales in 2021 and ranked number 102 in the Fortune 500 list of the largest United States corporations by total revenue.[10] As of 2021[update], the company had approximately 95,000 employees and operations in more than 70 countries.[11] There are a few international subsidiaries, such as 3M India, 3M Japan, and 3M Canada.[12]
In June 2023, 3M reached a settlement to pay more than $10 bill

KeyboardInterrupt: 